In [1]:
import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torch.autograd import Variable
from torch.optim import lr_scheduler

from sklearn.metrics import r2_score
from sklearn import preprocessing
#import scipy.io as sio

from models import VAE,AEBase
from models import DNN
import numpy as np
import pandas as pd
import models
import utils as ut
import copy



In D:\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In D:\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In D:\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In D:\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In D:\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The savefig.jpeg_quality rcparam was deprecated in M

In [2]:
from scipy.stats import pearsonr

# Parameters

In [3]:
# Define parameters
epochs = 500 #200,500,1000  
#dim_au_in = 20049
dim_au_out = 512 #8, 16, 32, 64, 128, 256,512
dim_dnn_in = dim_au_out
dim_dnn_out=1
select_drug = 'Gefitinib'
na = -1

# Import data

In [4]:
data_r=pd.read_csv('data/GDSCexpression.csv',index_col=0)
label_r=pd.read_csv('data/GDSClabel.csv',index_col=0)

In [5]:
label_r=label_r.fillna(na)

In [6]:
hvg,adata = ut.highly_variable_genes(data_r)

Observation names are not unique. To make them unique, call `.obs_names_make_unique`.


In [7]:
label_r

,Erlotinib,AICAR,Camptothecin,Vinblastine,Cisplatin,Cytarabine,Docetaxel,Methotrexate,ATRA,Gefitinib,...,CMK,Pyrimethamine,JW-7-52-1,A-443654,GW843682X,MS-275,Parthenolide,MG-132,KIN001-135,TGX221
BxPC-3,-1.000000,0.159708,0.188393,0.436127,0.154274,0.113764,0.249987,0.015324,0.015324,0.020293,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
KMOE-2,0.002108,0.339814,0.318206,0.281740,0.063808,0.140681,0.101530,0.191210,0.031229,0.007419,...,0.022662,0.025217,0.270791,0.095608,0.139914,0.171747,0.026670,0.013904,0.002116,0.000956
MFM-223,0.002148,0.334087,0.122671,0.361438,0.051983,0.186659,0.228797,0.003639,0.023043,0.003639,...,0.038997,0.006167,0.002185,0.102592,0.020338,0.010874,0.000970,0.035206,0.007051,0.000323
NUGC-3,-1.000000,0.095099,0.416792,0.422631,0.266409,0.237811,0.234952,0.011558,0.028838,0.153002,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
OC-314,-1.000000,0.012740,0.479915,0.390145,0.067738,0.125869,0.249668,0.110418,0.002214,0.020309,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
KP-N-S19s,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
BC-3,0.003515,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,...,0.048227,0.146476,0.400087,0.205247,0.147090,0.298803,0.023796,0.017219,0.002065,0.069181
Panc 08.13,-1.000000,0.152818,0.196279,0.180653,0.090963,0.093197,0.097267,0.018269,0.010237,0.009405,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
EKVX,0.177238,0.078395,0.078938,0.072213,0.007225,0.003470,0.129184,0.095642,0.111598,0.126364,...,0.020472,0.044047,0.243635,0.079375,0.019859,0.177759,0.014843,0.114431,0.007216,0.063911


In [8]:
selected_idx = label_r.loc[:,select_drug]!=na

In [9]:
data_r.columns = adata.var_names

# Your is gene-cell, mine is cell-gene

In [10]:
data = data_r.loc[selected_idx,:]
#data = data_r.loc[selected_idx,hvg]

In [11]:
label = label_r.loc[selected_idx,select_drug]
#sscaler = preprocessing.StandardScaler(with_mean=True, with_std=True)
mmscaler = preprocessing.MinMaxScaler()

data = mmscaler.fit_transform(data)
#data = sscaler.fit_transform(data)


In [12]:
print(np.std(data))
print(np.mean(data))

0.22376755104184073
0.33693391083156693


In [13]:
data.mean(axis=0)


array([0.49458949, 0.09687665, 0.50474807, ..., 0.20001366, 0.27789963,
       0.31386817])

In [14]:
print(data.max())
print(data.min())

1.0000000000000009
0.0


In [15]:
data.shape

(675, 11833)

In [16]:
label_r.shape

(789, 139)

# Split test train

In [17]:
from sklearn.model_selection import train_test_split
X_train_all, X_test, Y_train_all, Y_test = train_test_split(data, label, test_size=0.2, random_state=42)
X_train, X_valid, Y_train, Y_valid = train_test_split(X_train_all, Y_train_all, test_size=0.2, random_state=42)

In [18]:
print(data.shape)
print(label.shape)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

(675, 11833)
(675,)
(432, 11833) (432,)
(135, 11833) (135,)


In [19]:
print(X_train.max())
print(X_train.min())

1.0000000000000009
0.0


# AE MODEL

In [20]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Assuming that we are on a CUDA machine, this should print a CUDA device:
print(device)
torch.cuda.set_device(device)

cuda:0


# Add all data to AE

In [21]:
X_trainTensor = torch.FloatTensor(X_train).to(device)
X_validTensor = torch.FloatTensor(X_valid).to(device)
X_testTensor = torch.FloatTensor(X_test).to(device)
X_allTensor = torch.FloatTensor(data).to(device)
#X_alltrainTensor = torch.FloatTensor(X_train_all).to(device)


Y_trainTensor = torch.FloatTensor(Y_train.values).to(device)
Y_validTensor = torch.FloatTensor(Y_valid.values).to(device)

# construct TensorDataset
train_dataset = TensorDataset(X_trainTensor, X_trainTensor)
valid_dataset = TensorDataset(X_validTensor, X_validTensor)
test_dataset = TensorDataset(X_testTensor, X_testTensor)
all_dataset = TensorDataset(X_allTensor, X_allTensor)

X_trainDataLoader = DataLoader(dataset=train_dataset, batch_size=200, shuffle=True)
X_validDataLoader = DataLoader(dataset=valid_dataset, batch_size=200, shuffle=True)
X_allDataLoader = DataLoader(dataset=all_dataset, batch_size=200, shuffle=True)

In [22]:
dataloader = X_trainDataLoader

In [23]:
dataloaders_train = {'train':X_trainDataLoader,'val':X_validDataLoader}

In [24]:
X_trainDataLoader.dataset.tensors[0].shape[0]

432

In [25]:
{x: dataloaders_train[x].dataset.tensors[0].shape[0] for x in ['train', 'val']}

{'train': 432, 'val': 108}

# The model

In [26]:
model = AEBase(input_dim=data.shape[1],latent_dim=512,hidden_dims=[2048,1024])

In [27]:
print(model)

AEBase(
  (encoder): Sequential(
    (0): Sequential(
      (0): Linear(in_features=11833, out_features=2048, bias=True)
      (1): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): Dropout(p=0.3, inplace=False)
    )
    (1): Sequential(
      (0): Linear(in_features=2048, out_features=1024, bias=True)
      (1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): Dropout(p=0.3, inplace=False)
    )
  )
  (bottleneck): Linear(in_features=1024, out_features=512, bias=True)
  (decoder_input): Linear(in_features=512, out_features=1024, bias=True)
  (decoder): Sequential(
    (0): Sequential(
      (0): Linear(in_features=1024, out_features=2048, bias=True)
      (1): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): Dropout(p=0.3, inplace=False)
    )
  )
  (final_layer): Sequential(
    (0): Linear(in_features=2048, out_features=11833, bias=True)
    (1): Sigmoi

In [28]:
#model = VAE(dim_au_in=data_r.shape[1],dim_au_out=128)
if torch.cuda.is_available():
    model.cuda()
model.to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-2)
loss_function = nn.MSELoss()

# Decay LR by a factor of 0.1 every 7 epochs
#exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
exp_lr_scheduler = lr_scheduler.ReduceLROnPlateau(optimizer)

In [36]:
def train_ae_model(net,data_loaders,optimizer,loss_function,n_epochs,scheduler):
    
    dataset_sizes = {x: data_loaders[x].dataset.tensors[0].shape[0] for x in ['train', 'val']}
    loss_train = {}
    
    best_model_wts = copy.deepcopy(net.state_dict())
    best_loss = np.inf

    for epoch in range(n_epochs):
        print('Epoch {}/{}'.format(epoch, n_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                #optimizer = scheduler(optimizer, epoch)
                net.train()  # Set model to training mode
            else:
                net.eval()  # Set model to evaluate mode

            running_loss = 0.0

            # Iterate over data.
            # for data in data_loaders[phase]:
            for batchidx, (x, _) in enumerate(data_loaders[phase]):

                x.requires_grad_(True)
                # encode and decode 
                output = model(x)
                # compute loss
                loss = loss_function(output, x)      

                # zero the parameter (weight) gradients
                optimizer.zero_grad()

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    # update the weights
                    optimizer.step()

                # print loss statistics
                running_loss += loss.item()
            
            # Schedular
#             if phase == 'train':
#                 scheduler.step()
                
            epoch_loss = running_loss / dataset_sizes[phase]
            
            if phase == 'train':
                scheduler.step(epoch_loss)
                
            last_lr = scheduler.optimizer.param_groups[0]['lr']
            loss_train[epoch,phase] = epoch_loss
            print('{} Loss: {:.8f}. Learning rate = {}'.format(phase, epoch_loss,last_lr))
            
            if phase == 'val' and epoch_loss < best_loss:
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())
    
    # Select best model wts
    torch.save(best_model_wts, 'saved/models/ae.pkl')
    net.load_state_dict(best_model_wts)           
    
    return net, loss_train

In [37]:
model,loss_report = train_ae_model(net=model,data_loaders=dataloaders_train,
                             optimizer=optimizer,loss_function=loss_function,
                            n_epochs=epochs,scheduler=exp_lr_scheduler)

Epoch 0/499
----------
train Loss: 0.00124929. Learning rate = 0.01
val Loss: 0.00305093. Learning rate = 0.01
Epoch 1/499
----------
train Loss: 0.00109289. Learning rate = 0.01
val Loss: 0.00248298. Learning rate = 0.01
Epoch 2/499
----------
train Loss: 0.00096754. Learning rate = 0.01
val Loss: 0.00213574. Learning rate = 0.01
Epoch 3/499
----------
train Loss: 0.00090810. Learning rate = 0.01
val Loss: 0.00181604. Learning rate = 0.01
Epoch 4/499
----------
train Loss: 0.00096537. Learning rate = 0.01
val Loss: 0.00135841. Learning rate = 0.01
Epoch 5/499
----------
train Loss: 0.00090909. Learning rate = 0.01
val Loss: 0.00138355. Learning rate = 0.01
Epoch 6/499
----------
train Loss: 0.00085292. Learning rate = 0.01
val Loss: 0.00128438. Learning rate = 0.01
Epoch 7/499
----------
train Loss: 0.00084545. Learning rate = 0.01
val Loss: 0.00115831. Learning rate = 0.01
Epoch 8/499
----------
train Loss: 0.00077238. Learning rate = 0.01
val Loss: 0.00109419. Learning rate = 0.01
E

train Loss: 0.00016383. Learning rate = 0.01
val Loss: 0.00019560. Learning rate = 0.01
Epoch 75/499
----------
train Loss: 0.00016233. Learning rate = 0.01
val Loss: 0.00019241. Learning rate = 0.01
Epoch 76/499
----------
train Loss: 0.00015574. Learning rate = 0.01
val Loss: 0.00019378. Learning rate = 0.01
Epoch 77/499
----------
train Loss: 0.00015734. Learning rate = 0.01
val Loss: 0.00018968. Learning rate = 0.01
Epoch 78/499
----------
train Loss: 0.00015724. Learning rate = 0.01
val Loss: 0.00019272. Learning rate = 0.01
Epoch 79/499
----------
train Loss: 0.00015704. Learning rate = 0.01
val Loss: 0.00018954. Learning rate = 0.01
Epoch 80/499
----------
train Loss: 0.00015207. Learning rate = 0.01
val Loss: 0.00018895. Learning rate = 0.01
Epoch 81/499
----------
train Loss: 0.00015123. Learning rate = 0.01
val Loss: 0.00018773. Learning rate = 0.01
Epoch 82/499
----------
train Loss: 0.00015116. Learning rate = 0.01
val Loss: 0.00018618. Learning rate = 0.01
Epoch 83/499
---

train Loss: 0.00012488. Learning rate = 0.001
val Loss: 0.00016661. Learning rate = 0.001
Epoch 149/499
----------
train Loss: 0.00012246. Learning rate = 0.001
val Loss: 0.00016660. Learning rate = 0.001
Epoch 150/499
----------
train Loss: 0.00012455. Learning rate = 0.001
val Loss: 0.00016662. Learning rate = 0.001
Epoch 151/499
----------
train Loss: 0.00012027. Learning rate = 0.001
val Loss: 0.00016657. Learning rate = 0.001
Epoch 152/499
----------
train Loss: 0.00012384. Learning rate = 0.001
val Loss: 0.00016652. Learning rate = 0.001
Epoch 153/499
----------
train Loss: 0.00012251. Learning rate = 0.001
val Loss: 0.00016650. Learning rate = 0.001
Epoch 154/499
----------
train Loss: 0.00012234. Learning rate = 0.001
val Loss: 0.00016649. Learning rate = 0.001
Epoch 155/499
----------
train Loss: 0.00012475. Learning rate = 0.001
val Loss: 0.00016656. Learning rate = 0.001
Epoch 156/499
----------
train Loss: 0.00012315. Learning rate = 0.001
val Loss: 0.00016652. Learning rat

train Loss: 0.00012389. Learning rate = 1.0000000000000002e-07
val Loss: 0.00016653. Learning rate = 1.0000000000000002e-07
Epoch 213/499
----------
train Loss: 0.00012634. Learning rate = 1.0000000000000002e-07
val Loss: 0.00016654. Learning rate = 1.0000000000000002e-07
Epoch 214/499
----------
train Loss: 0.00012263. Learning rate = 1.0000000000000002e-07
val Loss: 0.00016653. Learning rate = 1.0000000000000002e-07
Epoch 215/499
----------
train Loss: 0.00012397. Learning rate = 1.0000000000000002e-07
val Loss: 0.00016651. Learning rate = 1.0000000000000002e-07
Epoch 216/499
----------
train Loss: 0.00012500. Learning rate = 1.0000000000000002e-07
val Loss: 0.00016651. Learning rate = 1.0000000000000002e-07
Epoch 217/499
----------
train Loss: 0.00012053. Learning rate = 1.0000000000000004e-08
val Loss: 0.00016650. Learning rate = 1.0000000000000004e-08
Epoch 218/499
----------
train Loss: 0.00012477. Learning rate = 1.0000000000000004e-08
val Loss: 0.00016653. Learning rate = 1.000

train Loss: 0.00012348. Learning rate = 1.0000000000000004e-08
val Loss: 0.00016654. Learning rate = 1.0000000000000004e-08
Epoch 269/499
----------
train Loss: 0.00012235. Learning rate = 1.0000000000000004e-08
val Loss: 0.00016658. Learning rate = 1.0000000000000004e-08
Epoch 270/499
----------
train Loss: 0.00012247. Learning rate = 1.0000000000000004e-08
val Loss: 0.00016656. Learning rate = 1.0000000000000004e-08
Epoch 271/499
----------
train Loss: 0.00012341. Learning rate = 1.0000000000000004e-08
val Loss: 0.00016654. Learning rate = 1.0000000000000004e-08
Epoch 272/499
----------
train Loss: 0.00012215. Learning rate = 1.0000000000000004e-08
val Loss: 0.00016652. Learning rate = 1.0000000000000004e-08
Epoch 273/499
----------
train Loss: 0.00012450. Learning rate = 1.0000000000000004e-08
val Loss: 0.00016650. Learning rate = 1.0000000000000004e-08
Epoch 274/499
----------
train Loss: 0.00012438. Learning rate = 1.0000000000000004e-08
val Loss: 0.00016660. Learning rate = 1.000

train Loss: 0.00012518. Learning rate = 1.0000000000000004e-08
val Loss: 0.00016657. Learning rate = 1.0000000000000004e-08
Epoch 325/499
----------
train Loss: 0.00012299. Learning rate = 1.0000000000000004e-08
val Loss: 0.00016662. Learning rate = 1.0000000000000004e-08
Epoch 326/499
----------
train Loss: 0.00012340. Learning rate = 1.0000000000000004e-08
val Loss: 0.00016655. Learning rate = 1.0000000000000004e-08
Epoch 327/499
----------
train Loss: 0.00012533. Learning rate = 1.0000000000000004e-08
val Loss: 0.00016653. Learning rate = 1.0000000000000004e-08
Epoch 328/499
----------
train Loss: 0.00012637. Learning rate = 1.0000000000000004e-08
val Loss: 0.00016656. Learning rate = 1.0000000000000004e-08
Epoch 329/499
----------
train Loss: 0.00012279. Learning rate = 1.0000000000000004e-08
val Loss: 0.00016652. Learning rate = 1.0000000000000004e-08
Epoch 330/499
----------
train Loss: 0.00012261. Learning rate = 1.0000000000000004e-08
val Loss: 0.00016654. Learning rate = 1.000

train Loss: 0.00012142. Learning rate = 1.0000000000000004e-08
val Loss: 0.00016657. Learning rate = 1.0000000000000004e-08
Epoch 381/499
----------
train Loss: 0.00012328. Learning rate = 1.0000000000000004e-08
val Loss: 0.00016660. Learning rate = 1.0000000000000004e-08
Epoch 382/499
----------
train Loss: 0.00012142. Learning rate = 1.0000000000000004e-08
val Loss: 0.00016655. Learning rate = 1.0000000000000004e-08
Epoch 383/499
----------
train Loss: 0.00012408. Learning rate = 1.0000000000000004e-08
val Loss: 0.00016655. Learning rate = 1.0000000000000004e-08
Epoch 384/499
----------
train Loss: 0.00012464. Learning rate = 1.0000000000000004e-08
val Loss: 0.00016659. Learning rate = 1.0000000000000004e-08
Epoch 385/499
----------
train Loss: 0.00012453. Learning rate = 1.0000000000000004e-08
val Loss: 0.00016656. Learning rate = 1.0000000000000004e-08
Epoch 386/499
----------
train Loss: 0.00012118. Learning rate = 1.0000000000000004e-08
val Loss: 0.00016653. Learning rate = 1.000

train Loss: 0.00012374. Learning rate = 1.0000000000000004e-08
val Loss: 0.00016653. Learning rate = 1.0000000000000004e-08
Epoch 437/499
----------
train Loss: 0.00012717. Learning rate = 1.0000000000000004e-08
val Loss: 0.00016669. Learning rate = 1.0000000000000004e-08
Epoch 438/499
----------
train Loss: 0.00012225. Learning rate = 1.0000000000000004e-08
val Loss: 0.00016663. Learning rate = 1.0000000000000004e-08
Epoch 439/499
----------
train Loss: 0.00012264. Learning rate = 1.0000000000000004e-08
val Loss: 0.00016663. Learning rate = 1.0000000000000004e-08
Epoch 440/499
----------
train Loss: 0.00012100. Learning rate = 1.0000000000000004e-08
val Loss: 0.00016662. Learning rate = 1.0000000000000004e-08
Epoch 441/499
----------
train Loss: 0.00012254. Learning rate = 1.0000000000000004e-08
val Loss: 0.00016658. Learning rate = 1.0000000000000004e-08
Epoch 442/499
----------
train Loss: 0.00012278. Learning rate = 1.0000000000000004e-08
val Loss: 0.00016655. Learning rate = 1.000

train Loss: 0.00012514. Learning rate = 1.0000000000000004e-08
val Loss: 0.00016657. Learning rate = 1.0000000000000004e-08
Epoch 493/499
----------
train Loss: 0.00012231. Learning rate = 1.0000000000000004e-08
val Loss: 0.00016651. Learning rate = 1.0000000000000004e-08
Epoch 494/499
----------
train Loss: 0.00012273. Learning rate = 1.0000000000000004e-08
val Loss: 0.00016653. Learning rate = 1.0000000000000004e-08
Epoch 495/499
----------
train Loss: 0.00012317. Learning rate = 1.0000000000000004e-08
val Loss: 0.00016654. Learning rate = 1.0000000000000004e-08
Epoch 496/499
----------
train Loss: 0.00012458. Learning rate = 1.0000000000000004e-08
val Loss: 0.00016656. Learning rate = 1.0000000000000004e-08
Epoch 497/499
----------
train Loss: 0.00012193. Learning rate = 1.0000000000000004e-08
val Loss: 0.00016657. Learning rate = 1.0000000000000004e-08
Epoch 498/499
----------
train Loss: 0.00012401. Learning rate = 1.0000000000000004e-08
val Loss: 0.00016656. Learning rate = 1.000

In [38]:
loss_report

{(0, 'train'): 0.001249286311644095,
 (0, 'val'): 0.003050928314526876,
 (1, 'train'): 0.0010928946581703644,
 (1, 'val'): 0.0024829818694679824,
 (2, 'train'): 0.0009675351557908234,
 (2, 'val'): 0.002135741627878613,
 (3, 'train'): 0.0009080959070059988,
 (3, 'val'): 0.0018160409397549098,
 (4, 'train'): 0.0009653714864894196,
 (4, 'val'): 0.0013584134479363759,
 (5, 'train'): 0.0009090929913024107,
 (5, 'val'): 0.0013835519828178265,
 (6, 'train'): 0.0008529184765561864,
 (6, 'val'): 0.0012843839272304817,
 (7, 'train'): 0.0008454506664916322,
 (7, 'val'): 0.0011583063061590547,
 (8, 'train'): 0.0007723803218040201,
 (8, 'val'): 0.0010941891620556514,
 (9, 'train'): 0.0007901015622472321,
 (9, 'val'): 0.0009912529753314124,
 (10, 'train'): 0.0007040542722852142,
 (10, 'val'): 0.0009728924681742986,
 (11, 'train'): 0.0007248849401043521,
 (11, 'val'): 0.0009006728866585979,
 (12, 'train'): 0.0006699894761873616,
 (12, 'val'): 0.0008894312023012726,
 (13, 'train'): 0.00066323623199153

In [ ]:
# # Old code for autoencoders
# for epoch in range(epochs):
#     # 不需要label，所以用一个占位符"_"代替
#     for batchidx, (x, _) in enumerate(X_allDataLoader):
#         x.requires_grad_(True)
#         # encode and decode 
#         output = model(x)
#         # compute loss
#         print(output.shape)
#         loss = loss_function(output, x)      
#         # update
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
           
#     loss_train[epoch,0] = loss.item()  
#     print('Epoch: %04d, Training loss=%.8f' %
#           (epoch+1, loss.item()))

In [ ]:
# torch.save(model.state_dict(), 'saved/models/ae.pkl')

In [39]:
recon_batch = model(X_trainTensor)

In [40]:
loss_function(recon_batch,X_trainTensor)

tensor(0.0174, device='cuda:0', grad_fn=<MseLossBackward>)

In [41]:
recon_batch

tensor([[0.5140, 0.0913, 0.5288,  ..., 0.2051, 0.3364, 0.3238],
        [0.5580, 0.1173, 0.4742,  ..., 0.2005, 0.3032, 0.3639],
        [0.0429, 0.1376, 0.3726,  ..., 0.1790, 0.3363, 0.3920],
        ...,
        [0.5027, 0.0823, 0.4501,  ..., 0.1798, 0.2450, 0.2799],
        [0.1499, 0.1126, 0.5041,  ..., 0.2068, 0.3374, 0.3552],
        [0.6287, 0.0781, 0.6299,  ..., 0.2292, 0.2406, 0.2678]],
       device='cuda:0', grad_fn=<SigmoidBackward>)

In [42]:
X_trainTensor

tensor([[0.7359, 0.0469, 0.5691,  ..., 0.1282, 0.3710, 0.2462],
        [0.5435, 0.1483, 0.4982,  ..., 0.3278, 0.3599, 0.4306],
        [0.1336, 0.0690, 0.3926,  ..., 0.2639, 0.3219, 0.1819],
        ...,
        [0.3869, 0.0385, 0.4286,  ..., 0.1089, 0.2763, 0.2801],
        [0.1415, 0.0447, 0.4723,  ..., 0.1098, 0.2252, 0.4184],
        [0.6655, 0.0802, 0.5544,  ..., 0.1544, 0.1192, 0.1983]],
       device='cuda:0')

In [43]:
recon_batch.cpu().detach().numpy().min()

0.0008136829

In [44]:
train_embeddings = model.encode(X_trainTensor)

In [45]:
feature = train_embeddings.cpu().detach().numpy()

In [46]:
from sklearn import linear_model
clf = linear_model.Lasso(alpha=0.001)
clf.fit(feature, Y_train.values)

D:\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.01398324966430664, tolerance: 0.00010032937279902399
  positive)


Lasso(alpha=0.001)

In [47]:
from sklearn.ensemble import RandomForestRegressor
RFrg = RandomForestRegressor(random_state=0, n_estimators=100)
RFrg.fit(feature, Y_train.values)

RandomForestRegressor(random_state=0)

In [48]:
testFeature = model.encode(X_testTensor)
lasso = clf.predict(testFeature.detach().cpu().numpy())

In [49]:
rfresult = RFrg.predict(testFeature.detach().cpu().numpy())

In [50]:
r2_score(lasso,Y_test)

-8.917931343703417

In [51]:
r2_score(rfresult,Y_test)

-2.09862380052972

In [52]:
Y_test.values

array([0.010893, 0.003969, 0.126364, 0.04815 , 0.013301, 0.003975,
       0.006658, 0.236411, 0.024986, 0.112077, 0.004105, 0.007248,
       0.015496, 0.079042, 0.066385, 0.006305, 0.006078, 0.025742,
       0.030093, 0.024503, 0.011125, 0.037697, 0.012029, 0.003817,
       0.010667, 0.067386, 0.057037, 0.006329, 0.067224, 0.003564,
       0.186092, 0.108377, 0.101198, 0.029871, 0.02189 , 0.043306,
       0.12798 , 0.005638, 0.038664, 0.002193, 0.005611, 0.157488,
       0.007132, 0.006106, 0.008437, 0.031169, 0.054449, 0.006736,
       0.016306, 0.003624, 0.093631, 0.011592, 0.010451, 0.003869,
       0.011615, 0.189386, 0.003639, 0.023794, 0.020825, 0.036415,
       0.00255 , 0.005367, 0.018914, 0.013027, 0.185878, 0.003549,
       0.004383, 0.007288, 0.012238, 0.019928, 0.075604, 0.01131 ,
       0.006595, 0.00253 , 0.005594, 0.018333, 0.015419, 0.010771,
       0.008096, 0.03138 , 0.043866, 0.014501, 0.002152, 0.015858,
       0.142685, 0.134998, 0.006991, 0.049048, 0.033088, 0.005

In [53]:
rfresult

array([0.05835779, 0.04809723, 0.08393456, 0.01316745, 0.01641077,
       0.03210582, 0.02638387, 0.09814184, 0.05537106, 0.17477762,
       0.02174419, 0.01008483, 0.05913225, 0.02961461, 0.04120421,
       0.03671159, 0.03117072, 0.03934981, 0.01289991, 0.03020184,
       0.01180884, 0.05170509, 0.02744221, 0.06279885, 0.03629024,
       0.04566286, 0.04937772, 0.0138828 , 0.07842328, 0.01707409,
       0.05549958, 0.03840715, 0.03311106, 0.06620793, 0.04880786,
       0.01589267, 0.07003126, 0.02660924, 0.01285711, 0.00950247,
       0.01283124, 0.03617788, 0.01042021, 0.02128065, 0.07431409,
       0.04035205, 0.01197993, 0.09221617, 0.02525261, 0.01656044,
       0.02447865, 0.10361253, 0.01008788, 0.01298795, 0.0279641 ,
       0.06334959, 0.02276789, 0.01078643, 0.03309207, 0.01934929,
       0.0295273 , 0.03260531, 0.03851115, 0.02190465, 0.16168004,
       0.03004164, 0.02372988, 0.04582957, 0.01219899, 0.03269482,
       0.06253785, 0.04725378, 0.02765245, 0.03629509, 0.04448

In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
tsne_results = tsne.fit_transform(feature)

In [ ]:
tsne_results

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.scatter(tsne_results[:, 0], tsne_results[:, 1], label="label")
plt.legend()
plt.savefig("saved/figures/tsne_ae_gdsc.png")

In [ ]:
EPOCH = 500

In [ ]:
# Load data
# data type conversion
B_feature = torch.FloatTensor(feature).to(device)
y = torch.FloatTensor(Y_train.values).to(device)
# construct TensorDataset
b_data = TensorDataset(B_feature, y)
trainDataLoader2 = DataLoader(dataset=b_data, batch_size=200, shuffle=False)

In [ ]:
# Initialization DNN model

predictor = DNN(128, dim_dnn_out).to(device)
optimizer = optim.Adam(predictor.parameters(), lr=1e-3,betas=(0.9,0.99))
#loss1-softmax
loss_func = nn.MSELoss().to(device)
#loss2-sigmoid
#loss_func = nn.BCELoss()
#loss3-sigmoid
#loss_func = nn.CrossEntropyLoss()

#criterion = torch.nn.MSELoss(size_average=True)
#criterion = torch.nn.BCELoss(size_average=True) # Defined loss function
#optimizer = optim.Adm(model.parameters(), lr=0.01) # Defined optimizer
loss_train = np.zeros((epochs, 1))

In [ ]:
# train model
for epoch in range(EPOCH):
    print('Epoch: ',epoch)
    for step,(batch_x,batch_y) in enumerate(trainDataLoader2):
        b_x = Variable(batch_x)
        b_y = Variable(batch_y)
        # predict label
        output = predictor(b_x)
        # b_y=F.sigmoid(b_y) 
        
        #print
        #print(output)
        #print(b_y)
        # compute loss
        loss = loss_func(output,b_y)
        #loss = criterion(output, b_y)
        
        # update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    loss_train[epoch,0] = loss.item()  
    print('Epoch: %04d, Training loss=%.8f' %
          (epoch+1, loss.item())) 

# Save model
torch.save(predictor.state_dict(), 'saved/models/DNN_GDSC.pkl')

In [ ]:
testpredict = predictor(testFeature)

In [ ]:
testpredict

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
r2_score(testpredict.detach().cpu().numpy(),Y_test)

In [ ]:
mean_squared_error(testpredict.detach().cpu().numpy(),Y_test)